In [3]:
import pandas as pd
file=pd.read_csv(r"C:\Users\Hp\Downloads\purchase_data.csv")
data=file.copy()
data.head(10)

,Purchase_ID,User_ID,Product_ID,Transaction_Amount,Purchase_Time
0,Purchase_948,User_3840,Product_19,62.843031,2023-04-02 16:35:46
1,Purchase_905,User_1846,Product_9,21.780079,2023-04-14 16:38:27
2,Purchase_704,User_1700,Product_8,138.264455,2023-01-21 11:21:31
3,Purchase_766,User_4117,Product_15,74.991367,2023-01-29 04:44:04
4,Purchase_36,User_3053,Product_7,162.569998,2023-05-24 03:30:23
5,Purchase_982,User_3759,Product_16,49.945558,2023-01-29 22:34:04
6,Purchase_950,User_4573,Product_18,181.501507,2023-07-24 06:13:14
7,Purchase_599,User_3228,Product_12,46.880509,2023-03-22 01:31:48
8,Purchase_613,User_691,Product_11,104.729904,2023-06-19 08:35:13
9,Purchase_81,User_940,Product_19,98.921151,2023-03-06 06:43:11


In [74]:
print(data.columns)

Index(['Purchase_ID', 'User_ID', 'Product_ID', 'Transaction_Amount',
       'Purchase_Time'],
      dtype='object')


In [75]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Purchase_ID         1000 non-null   object        
 1   User_ID             1000 non-null   object        
 2   Product_ID          1000 non-null   object        
 3   Transaction_Amount  1000 non-null   float64       
 4   Purchase_Time       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 39.2+ KB
None


In [8]:
## checking for null entries
print(data.isnull().sum())
data.dropna(inplace=True)

Purchase_ID           0
User_ID               0
Product_ID            0
Transaction_Amount    0
Purchase_Time         0
dtype: int64


In [11]:
## checking for duplicated records
print(data.duplicated().sum())

0


In [73]:
print(data.shape)
print(cleaned_data.shape)

(1000, 5)
(1000, 5)


In [13]:
## removing outliers
import numpy as np
def remove_outliers_iqr(data,column, k=1.5):   ## k standard value=1.5

    q1 = np.percentile(data[column], 25)
    q3 = np.percentile(data[column], 75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    cond1=data[column] >= lower_bound
    cond2=(data[column] <= upper_bound)
    return data[cond1 & cond2]
cleaned_data=remove_outliers_iqr(data,'Transaction_Amount',k=1.5)
print(cleaned_data)

      Purchase_ID    User_ID  Product_ID  Transaction_Amount  \
0    Purchase_948  User_3840  Product_19           62.843031   
1    Purchase_905  User_1846   Product_9           21.780079   
2    Purchase_704  User_1700   Product_8          138.264455   
3    Purchase_766  User_4117  Product_15           74.991367   
4     Purchase_36  User_3053   Product_7          162.569998   
..            ...        ...         ...                 ...   
995  Purchase_489  User_2879  Product_14           47.047885   
996  Purchase_741   User_588   Product_1           85.587089   
997  Purchase_587  User_3821  Product_16          162.919467   
998  Purchase_222  User_1804  Product_10           44.435545   
999  Purchase_609  User_3364  Product_14          147.367700   

           Purchase_Time  
0    2023-04-02 16:35:46  
1    2023-04-14 16:38:27  
2    2023-01-21 11:21:31  
3    2023-01-29 04:44:04  
4    2023-05-24 03:30:23  
..                   ...  
995  2023-04-10 17:38:54  
996  2023-02-19

In [14]:
## this shows that there are no outliers
cleaned_data.shape

(1000, 5)

In [20]:
## overall conversion rate

file2=pd.read_csv(r"C:\Users\Hp\Downloads\user_interactions.csv")
user_data=file2.copy()
user_data.drop(user_data.index[user_data.duplicated()],inplace=True)
Total_conversions=user_data.shape[0]

Total_successful_conversions=cleaned_data.shape[0]

overall_conversion_rate=Total_successful_conversions/Total_conversions
print("Overall Convertion Rate:",overall_conversion_rate)

Overall Convertion Rate: 0.20004000800160032


In [35]:
## top selling product

grouped= cleaned_data.groupby('Product_ID')['Purchase_ID'].count()
Top_selling_product=grouped.idxmax()
count=grouped.max()
print("Top Selling Product's Product_ID:",Top_selling_product)
print("Top Selling Product Count:",count)

Top Selling Product's Product_ID: Product_19
Top Selling Product Count: 66


In [53]:
## Relationship between number of interactions on the website and likelihood of the purchase

import pandas as pd

user_interactions_data = pd.read_csv(r"C:\Users\Hp\Downloads\user_interactions.csv")
purchase_data = pd.read_csv(r"C:\Users\Hp\Downloads\purchase_data.csv")

# Calculating total interactions (page views) per user
total_interactions_per_user = user_interactions_data.groupby('User_ID')['Page_Viewed'].count().reset_index()
total_interactions_per_user.rename(columns={'Page_Viewed': 'Total_Interactions'}, inplace=True)

# Calculating total purchases per user
total_purchases_per_user = purchase_data.groupby('User_ID')['Purchase_ID'].count().reset_index()
total_purchases_per_user.rename(columns={'Purchase_ID': 'Total_Purchases'}, inplace=True)

# Merge the DataFrames based on 'User_ID'
merged_data = pd.merge(total_interactions_per_user, total_purchases_per_user, on='User_ID', how='left')

# Calculating the conversion rate (likelihood of a purchase) for each user
merged_data['Conversion_Rate'] = (merged_data['Total_Purchases'] / merged_data['Total_Interactions']) * 100
merged_data.head(50)

,User_ID,Total_Interactions,Total_Purchases,Conversion_Rate
0,User_1,2,NaN,NaN
1,User_100,1,NaN,NaN
2,User_1000,1,1.0,100.0
3,User_1002,1,NaN,NaN
4,User_1004,1,NaN,NaN
5,User_1005,3,NaN,NaN
6,User_1006,1,NaN,NaN
7,User_1008,1,NaN,NaN
8,User_1009,2,NaN,NaN
9,User_101,1,NaN,NaN


In [55]:
merge_data.shape[0]

1029